In [30]:
#import libraries
import pandas as pd  #for data handeling
import numpy as np  #random number generation
import random  #for random choices
from datetime import datetime, timedelta #date, time manipulation
!pip install openpyxl



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
from faker import Faker

In [66]:
#creating an instance Faker package
synthetic = Faker(locale='en_GB')

In [32]:
#function to generate date randomly that falls between start and end range given
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

In [145]:
#Generate demographics data

# Function to generate pseudoID in sorted order
def generate_pseudoID(index):
    return f'ID_{index:04}'  # ID_0000, ID_0001, ...

# Number of records to generate
num_records = 3000

# Generate data
data = {
    'PPID': [generate_pseudoID(index) for index in range(num_records)],
    'Sex': np.random.choice(['M', 'F'], num_records, p=[0.48, 0.52]), 
    'MoB': np.random.choice(range(1,13), num_records), #randomly selects 1-12 inclusive representing months of the year
    'YoB': np.random.randint(1983,2002, num_records), #generates random years btween 1983-2022
    'DoD': [random_date(datetime(2004,1,1), datetime(2023,5,21)) 
            if np.random.rand()<0.1 else None
            for _ in range(num_records)], #randomly assign death date to 10% of data, NULL if no death date
    'Ethnicity6': np.random.choice(['White','Asian','Black','others','not_stated'], num_records),
}
#create dataframe
demog = pd.DataFrame(data)
print(demog.head())

      PPID Sex  MoB   YoB        DoD  Ethnicity6
0  ID_0000   M    9  1987 2012-08-08  not_stated
1  ID_0001   F   12  1991        NaT       White
2  ID_0002   M   12  1993        NaT       White
3  ID_0003   M    3  2000        NaT      others
4  ID_0004   F    2  1988        NaT      others


In [146]:
 #randomly choosing dod
demog['DoD']= [np.random.choice(['-',synthetic.date_this_decade(before_today=True),],
                                 p=[0.80,0.20]) for _ in range(num_records)]
        

In [147]:
# Count the number of records with a date of death
dod_count = demog['DoD'].value_counts()
# Calculate the proportion of records with a date of death
percentage_with_dod = dod_count.loc[dod_count.index != '-'].sum() / num_records * 100

# Print the results
print(f"Total records: {num_records}")
print(f"Number of records with a date of death: {dod_count.loc[dod_count.index != '-'].sum()}")
print(f"Percentage of records with a date of death: {percentage_with_dod:.2f}%")

print(demog.head())

Total records: 3000
Number of records with a date of death: 574
Percentage of records with a date of death: 19.13%
      PPID Sex  MoB   YoB DoD  Ethnicity6
0  ID_0000   M    9  1987   -  not_stated
1  ID_0001   F   12  1991   -       White
2  ID_0002   M   12  1993   -       White
3  ID_0003   M    3  2000   -      others
4  ID_0004   F    2  1988   -      others


In [148]:
##generate GP registration date
##minimum GP reg date is btween 1893/10/27 to 2023/05/18
## however date of registration should be btween date of birth and 2023/05/18

demog['Reg_Start_Date'] = [
        random_date(datetime(demog['YoB'][i], 1, 1), datetime(2005, 1, 1))
        for i in range(num_records)
    ]
print(demog.head())


      PPID Sex  MoB   YoB DoD  Ethnicity6 Reg_Start_Date
0  ID_0000   M    9  1987   -  not_stated     1988-09-30
1  ID_0001   F   12  1991   -       White     1994-11-26
2  ID_0002   M   12  1993   -       White     1999-07-19
3  ID_0003   M    3  2000   -      others     2002-12-05
4  ID_0004   F    2  1988   -      others     1996-05-19


In [149]:
# Assign end dates to 30% of the records
reg_end_dates = []
for i in range(num_records):
    if demog['DoD'][i]:
        reg_end_dates.append(demog['DoD'][i]) #check if record has date of death and assign that as the end date
    elif np.random.rand() < 0.30:
        reg_end_dates.append(random_date(demog['Min_Reg_Start_Date'][i], datetime(2023, 5, 18)))
    else:
        reg_end_dates.append(None)

demog['Reg_End_Date'] = reg_end_dates
print(demog.head())

      PPID Sex  MoB   YoB DoD  Ethnicity6 Reg_Start_Date Reg_End_Date
0  ID_0000   M    9  1987   -  not_stated     1988-09-30            -
1  ID_0001   F   12  1991   -       White     1994-11-26            -
2  ID_0002   M   12  1993   -       White     1999-07-19            -
3  ID_0003   M    3  2000   -      others     2002-12-05            -
4  ID_0004   F    2  1988   -      others     1996-05-19            -


In [150]:
# Count the number of records with a date of death
count = demog['Reg_Start_Date'].value_counts()
# Calculate the proportion of records with a date of death
percentage = count.loc[count.index != '-'].sum() / num_records * 100

# Print the results
print(f"Total records: {num_records}")
print(f"Number of records: {count.loc[count.index != '-'].sum()}")
print(f"Percentage of records with a date of death: {percentage:.2f}%")

print(demog.head())

Total records: 3000
Number of records: 3000
Percentage of records with a date of death: 100.00%
      PPID Sex  MoB   YoB DoD  Ethnicity6 Reg_Start_Date Reg_End_Date
0  ID_0000   M    9  1987   -  not_stated     1988-09-30            -
1  ID_0001   F   12  1991   -       White     1994-11-26            -
2  ID_0002   M   12  1993   -       White     1999-07-19            -
3  ID_0003   M    3  2000   -      others     2002-12-05            -
4  ID_0004   F    2  1988   -      others     1996-05-19            -


In [151]:
# Assign GP Follow-Up End date
demog['GP_Follow_Up_End'] = [
    demog['DoD'][i] or demog['Reg_End_Date'][i] or datetime(2023, 5, 18)
    for i in range(num_records)
]

print(demog.head())

      PPID Sex  MoB   YoB DoD  Ethnicity6 Reg_Start_Date Reg_End_Date  \
0  ID_0000   M    9  1987   -  not_stated     1988-09-30            -   
1  ID_0001   F   12  1991   -       White     1994-11-26            -   
2  ID_0002   M   12  1993   -       White     1999-07-19            -   
3  ID_0003   M    3  2000   -      others     2002-12-05            -   
4  ID_0004   F    2  1988   -      others     1996-05-19            -   

  GP_Follow_Up_End  
0                -  
1                -  
2                -  
3                -  
4                -  


In [152]:
# Count
count = demog['GP_Follow_Up_End'].value_counts()
# Calculate the proportion of records with a date of death
percentage_with1 = count.loc[count.index != '-'].sum() / num_records * 100

# Print the results
print(f"Total records: {num_records}")
print(f"Number of records with a date of death: {count.loc[count.index != '-'].sum()}")
print(f"Percentage of records with a date of death: {percentage_with1:.2f}%")

print(demog.head())

Total records: 3000
Number of records with a date of death: 574
Percentage of records with a date of death: 19.13%
      PPID Sex  MoB   YoB DoD  Ethnicity6 Reg_Start_Date Reg_End_Date  \
0  ID_0000   M    9  1987   -  not_stated     1988-09-30            -   
1  ID_0001   F   12  1991   -       White     1994-11-26            -   
2  ID_0002   M   12  1993   -       White     1999-07-19            -   
3  ID_0003   M    3  2000   -      others     2002-12-05            -   
4  ID_0004   F    2  1988   -      others     1996-05-19            -   

  GP_Follow_Up_End  
0                -  
1                -  
2                -  
3                -  
4                -  


In [153]:
# Fill missing values in 'GP' column with datetime(2023, 5, 18)

# Fill missing values and '-' with datetime(2023, 5, 18)
demog['GP_Follow_Up_End'] = demog['GP_Follow_Up_End'].replace(['-', None], datetime(2023, 5, 18))

In [154]:
# Convert DataFrame to TSV format and save to a specific path
demog.to_csv('C:/Users/User/Desktop/demog_synthetic_data.tsv', sep='\t', index=False)


DIAGNOSIS

In [130]:
#create a dataframe with PPID and Reg start date
## for the diagnosis 

# Define the conditions
conditions = ['Asthma', 'COPD', 'Wheeze', 'ILD']

# Expand the DataFrame with multiple entries for some PPIDs
expanded_data = []

for _, row in demog.iterrows():
    num_entries = np.random.randint(1, 5)  # Randomly choose how many entries for each PPID betwn 1 to 4
    for _ in range(num_entries):
        condition = np.random.choice(conditions) #applies the codition randomly
        expanded_data.append({
            'PPID': row['PPID'],
            'Reg_Start_Date': row['Reg_Start_Date'],
            'Condition': condition
        })

# Create the expanded DataFrame
diagnosis = pd.DataFrame(expanded_data)
print(diagnosis)

         PPID Reg_Start_Date Condition
0     ID_0000     2003-10-01       ILD
1     ID_0000     2003-10-01    Wheeze
2     ID_0001     1991-03-19       ILD
3     ID_0001     1991-03-19       ILD
4     ID_0001     1991-03-19    Asthma
...       ...            ...       ...
7541  ID_2997     2005-10-09    Asthma
7542  ID_2997     2005-10-09      COPD
7543  ID_2997     2005-10-09      COPD
7544  ID_2998     1992-07-26    Wheeze
7545  ID_2999     2006-11-08      COPD

[7546 rows x 3 columns]


In [155]:
## on the dignosis data frame, add earliest mention date has to be between date registered and 2007
## if registered date is before 1990, then earliest mention date is between 1990 and 2007

 #function generates random dates betwn 1990 and 2007
def random_date(start, end):
    delta = end - start
    return start + timedelta(days=np.random.randint(0, delta.days + 1))

# Generate EarliestMention column
def generate_earliest_mention(Reg_Start_Date):
    if Reg_Start_Date.year < 1990:
        start_date = datetime(1990, 1, 1)
    else:
        start_date = Reg_Start_Date
    end_date = datetime(2007, 12, 31)
    return random_date(start_date, end_date)


diagnosis['EarliestMention'] = diagnosis['Reg_Start_Date'].apply(generate_earliest_mention) #apply to the dataframe each function


print(diagnosis)

         PPID Reg_Start_Date Condition EarliestMention EarliestIncidence  \
0     ID_0000     2003-10-01       ILD      2007-12-14        2007-04-14   
1     ID_0000     2003-10-01    Wheeze      2005-12-08        2007-10-30   
2     ID_0001     1991-03-19       ILD      2005-10-26        2007-07-07   
3     ID_0001     1991-03-19       ILD      2004-05-30        2007-11-07   
4     ID_0001     1991-03-19    Asthma      2004-01-13        2007-07-11   
...       ...            ...       ...             ...               ...   
7541  ID_2997     2005-10-09    Asthma      2006-03-04        2007-07-13   
7542  ID_2997     2005-10-09      COPD      2007-07-21        2007-08-15   
7543  ID_2997     2005-10-09      COPD      2006-03-09        2007-09-29   
7544  ID_2998     1992-07-26    Wheeze      2002-05-15        2007-04-21   
7545  ID_2999     2006-11-08      COPD      2006-11-15        2007-11-01   

     EarliestIncidencePrimaryCare  
0                      2007-04-14  
1              

In [157]:
#creating earliest incidence
# Function to generate EarliestIncidence
def generate_Earliest_Incidence(row):
    reg_start_date = row['Reg_Start_Date']
    earliest_mention = row['EarliestMention']
    start_date = reg_start_date + timedelta(days=365)  # 1 year after registration date
    end_date = earliest_mention

    if start_date >= end_date:
        # If start_date is not before end_date, set start_date to a day before end_date
        start_date = end_date - timedelta(days=1)

    return random_date(start_date, end_date)

# Add the EarliestIncidence column
diagnosis['EarliestIncidence'] = diagnosis.apply(generate_Earliest_Incidence, axis=1) #axis=1 ensures the function is applied to rows instead of columns
print(diagnosis)

         PPID Reg_Start_Date Condition EarliestMention EarliestIncidence  \
0     ID_0000     2003-10-01       ILD      2007-12-14        2007-09-28   
1     ID_0000     2003-10-01    Wheeze      2005-12-08        2005-09-19   
2     ID_0001     1991-03-19       ILD      2005-10-26        2003-10-24   
3     ID_0001     1991-03-19       ILD      2004-05-30        1994-04-10   
4     ID_0001     1991-03-19    Asthma      2004-01-13        2000-08-02   
...       ...            ...       ...             ...               ...   
7541  ID_2997     2005-10-09    Asthma      2006-03-04        2006-03-04   
7542  ID_2997     2005-10-09      COPD      2007-07-21        2006-10-16   
7543  ID_2997     2005-10-09      COPD      2006-03-09        2006-03-09   
7544  ID_2998     1992-07-26    Wheeze      2002-05-15        1997-05-07   
7545  ID_2999     2006-11-08      COPD      2006-11-15        2006-11-15   

     EarliestIncidencePrimaryCare  
0                      2007-04-14  
1              

In [159]:
##ealiest incidence in primary care, since all data gotten is form primary care this can be same as incidence date
#making all values in earliest incidence and earliest incideence in primary care same
diagnosis['EarliestIncidencePrimaryCare'] = diagnosis['EarliestIncidence']
print(diagnosis)

         PPID Reg_Start_Date Condition EarliestMention EarliestIncidence  \
0     ID_0000     2003-10-01       ILD      2007-12-14        2007-09-28   
1     ID_0000     2003-10-01    Wheeze      2005-12-08        2005-09-19   
2     ID_0001     1991-03-19       ILD      2005-10-26        2003-10-24   
3     ID_0001     1991-03-19       ILD      2004-05-30        1994-04-10   
4     ID_0001     1991-03-19    Asthma      2004-01-13        2000-08-02   
...       ...            ...       ...             ...               ...   
7541  ID_2997     2005-10-09    Asthma      2006-03-04        2006-03-04   
7542  ID_2997     2005-10-09      COPD      2007-07-21        2006-10-16   
7543  ID_2997     2005-10-09      COPD      2006-03-09        2006-03-09   
7544  ID_2998     1992-07-26    Wheeze      2002-05-15        1997-05-07   
7545  ID_2999     2006-11-08      COPD      2006-11-15        2006-11-15   

     EarliestIncidencePrimaryCare  
0                      2007-09-28  
1              

In [161]:
#randomly assign NULL
mask = np.random.rand(len(diagnosis)) < 0.07 #7% NULL is applied randomly
diagnosis.loc[mask, 'EarliestIncidencePrimaryCare'] = None
print(diagnosis)

         PPID Reg_Start_Date Condition EarliestMention EarliestIncidence  \
0     ID_0000     2003-10-01       ILD      2007-12-14        2007-09-28   
1     ID_0000     2003-10-01    Wheeze      2005-12-08        2005-09-19   
2     ID_0001     1991-03-19       ILD      2005-10-26        2003-10-24   
3     ID_0001     1991-03-19       ILD      2004-05-30        1994-04-10   
4     ID_0001     1991-03-19    Asthma      2004-01-13        2000-08-02   
...       ...            ...       ...             ...               ...   
7541  ID_2997     2005-10-09    Asthma      2006-03-04        2006-03-04   
7542  ID_2997     2005-10-09      COPD      2007-07-21        2006-10-16   
7543  ID_2997     2005-10-09      COPD      2006-03-09        2006-03-09   
7544  ID_2998     1992-07-26    Wheeze      2002-05-15        1997-05-07   
7545  ID_2999     2006-11-08      COPD      2006-11-15        2006-11-15   

     EarliestIncidencePrimaryCare EarliestIncidenceInPrimaryCare  
0                   

In [166]:
#count
count = diagnosis['EarliestIncidencePrimaryCare'].value_counts()
# Calculate the proportion of records with a date of death
percentage_with1 = count.loc[count.index != '-'].sum() / num_records * 100

# Print the results
print(f"Total records: {num_records}")
print(f"Number of records with a date of death: {count.loc[count.index != '-'].sum()}")
print(f"Percentage of records with a date of death: {percentage_with1:.2f}%")

print(diagnosis.head())

Total records: 3000
Number of records with a date of death: 6983
Percentage of records with a date of death: 232.77%
      PPID Reg_Start_Date Condition EarliestMention EarliestIncidence  \
0  ID_0000     2003-10-01       ILD      2007-12-14        2007-09-28   
1  ID_0000     2003-10-01    Wheeze      2005-12-08        2005-09-19   
2  ID_0001     1991-03-19       ILD      2005-10-26        2003-10-24   
3  ID_0001     1991-03-19       ILD      2004-05-30        1994-04-10   
4  ID_0001     1991-03-19    Asthma      2004-01-13        2000-08-02   

  EarliestIncidencePrimaryCare  
0                   2007-09-28  
1                   2005-09-19  
2                   2003-10-24  
3                   1994-04-10  
4                   2000-08-02  


In [165]:
#drop
diagnosis.drop(['EarliestIncidenceInPrimaryCare'],axis=1,inplace=True)

In [167]:
##earliest mention in primary care
##has to be same as earliest mention
diagnosis['EarliestMentionPrimaryCare'] = diagnosis['EarliestMention']
#randomly assign NULL
mask1 = np.random.rand(len(diagnosis)) < 0.07 #7% NULL is applied randomly
diagnosis.loc[mask1, 'EarliestMentionPrimaryCare'] = None
print(diagnosis)

         PPID Reg_Start_Date Condition EarliestMention EarliestIncidence  \
0     ID_0000     2003-10-01       ILD      2007-12-14        2007-09-28   
1     ID_0000     2003-10-01    Wheeze      2005-12-08        2005-09-19   
2     ID_0001     1991-03-19       ILD      2005-10-26        2003-10-24   
3     ID_0001     1991-03-19       ILD      2004-05-30        1994-04-10   
4     ID_0001     1991-03-19    Asthma      2004-01-13        2000-08-02   
...       ...            ...       ...             ...               ...   
7541  ID_2997     2005-10-09    Asthma      2006-03-04        2006-03-04   
7542  ID_2997     2005-10-09      COPD      2007-07-21        2006-10-16   
7543  ID_2997     2005-10-09      COPD      2006-03-09        2006-03-09   
7544  ID_2998     1992-07-26    Wheeze      2002-05-15        1997-05-07   
7545  ID_2999     2006-11-08      COPD      2006-11-15        2006-11-15   

     EarliestIncidencePrimaryCare EarliestMentionPrimaryCare  
0                      2

In [168]:
# Convert DataFrame to TSV format and save to a specific path
diagnosis.to_csv('C:/Users/User/Desktop/diagnosis_synthetic_data.tsv', sep='\t', index=False)

CONDITION EVENT

In [171]:
#PPID AND conditions from diagnosis
##events date: the date these events happen? random dates after YOB 

#take ppid, condition and reg start date

conditionEvent = diagnosis[['PPID','Reg_Start_Date', 'Condition']] #double square bracket for moving multiple column
print(conditionEvent)

         PPID Reg_Start_Date Condition
0     ID_0000     2003-10-01       ILD
1     ID_0000     2003-10-01    Wheeze
2     ID_0001     1991-03-19       ILD
3     ID_0001     1991-03-19       ILD
4     ID_0001     1991-03-19    Asthma
...       ...            ...       ...
7541  ID_2997     2005-10-09    Asthma
7542  ID_2997     2005-10-09      COPD
7543  ID_2997     2005-10-09      COPD
7544  ID_2998     1992-07-26    Wheeze
7545  ID_2999     2006-11-08      COPD

[7546 rows x 3 columns]


In [172]:
#to allocate code and description column
#first we map each condition with its code and description

condition_mapping = {
    'Asthma': [
        ('173A.00', 'Exercise induced asthma'),
        ('173c.00', 'Occupation asthma'),
        ('1780.00', 'Aspirin induced asthma'),
        ('1782.00', 'Asthma trigger-tobacco smoke'),
        ('178B.00', 'Asthma trigger-Exercise'),
        ('102.00', 'Asthma confirmed'),
        ('663V000', 'Occasional asthma'),
        ('663V200', 'Moderate asthma'),
        ('663V300', 'Severe asthma'),
        ('H33z100', 'Asthma attack')
    ],
    'COPD': [
        ('66Yg.00', 'COPD disturbs sleep'),
        ('66Yh.00', 'COPD does not disturb sleep'),
        ('H30Z.00', 'Bronchitis'),
        ('J410', 'Simple chronic bronchitis'),
        ('J43X', 'Emphysema'),
        ('J44X', 'Other COPD')
    ],
    'ILD': [
        ('AD50.00', 'Sarcoidosis of lung'),
        ('C370.11', 'Fibrocystic disease'),
        ('H350.00', 'Farmer’s lung'),
        ('H351.00', 'Bagassosis'),
        ('H41..00', 'Asbestosis'),
        ('H435.00', 'Stannosis'),
        ('H563200', 'Pulmonary fibrosis')
    ],
    'Wheeze': [
        ('1737.00', 'Wheezing'),
        ('1737.11', 'Wheezing symptoms'),
        ('1737100', 'Wheezing in absence of cold'),
        ('173e.00', 'Viral wheezing'),
        ('R060G00', 'Moderate wheezing'),
        ('R060H00', 'Severe wheezing')
    ]
}


In [174]:
#create function to asssign this code to individual rows with conditions
##this function takes each row, as an inpute and randomly returns a code and description according to thwe condition row
def assign_code_and_description(row):
    condition = row['Condition']
    if condition in condition_mapping:
        code_desc = condition_mapping[condition][np.random.randint(len(condition_mapping[condition]))]
        return pd.Series([code_desc[0], code_desc[1]], index=['Code', 'Description'])
    else:
        return pd.Series([np.nan, np.nan], index=['Code', 'Description'])

# Apply is used to apply the function while creating new columns  to each row in the DataFrame, axis 1 enures every row is dealt with
#
conditionEvent[['Code', 'Description']] = conditionEvent.apply(assign_code_and_description, axis=1)

print(conditionEvent)

         PPID Reg_Start_Date Condition     Code                Description
0     ID_0000     2003-10-01       ILD  H351.00                 Bagassosis
1     ID_0000     2003-10-01    Wheeze  1737.11          Wheezing symptoms
2     ID_0001     1991-03-19       ILD  H563200         Pulmonary fibrosis
3     ID_0001     1991-03-19       ILD  H350.00              Farmer’s lung
4     ID_0001     1991-03-19    Asthma  663V000          Occasional asthma
...       ...            ...       ...      ...                        ...
7541  ID_2997     2005-10-09    Asthma  173A.00    Exercise induced asthma
7542  ID_2997     2005-10-09      COPD     J410  Simple chronic bronchitis
7543  ID_2997     2005-10-09      COPD  66Yg.00        COPD disturbs sleep
7544  ID_2998     1992-07-26    Wheeze  1737.11          Wheezing symptoms
7545  ID_2999     2006-11-08      COPD     J43X                  Emphysema

[7546 rows x 5 columns]


C:\Users\user\AppData\Local\Temp\ipykernel_9628\1857652564.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conditionEvent[['Code', 'Description']] = conditionEvent.apply(assign_code_and_description, axis=1)
C:\Users\user\AppData\Local\Temp\ipykernel_9628\1857652564.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conditionEvent[['Code', 'Description']] = conditionEvent.apply(assign_code_and_description, axis=1)


In [176]:
max_date = conditionEvent['Reg_Start_Date'].max()
print(max_date)

2006-12-31 00:00:00


In [180]:
#setting event date
## any date from rege start date till 2007

def generate_event_date(Reg_Start_Date):
    start_date = Reg_Start_Date
    end_date = datetime(2007,1,1)
    return random_date(start_date,end_date)

conditionEvent.loc[:, 'Event_date'] = conditionEvent['Reg_Start_Date'].apply(generate_event_date)

print(conditionEvent)

         PPID Reg_Start_Date Condition     Code                Description  \
0     ID_0000     2003-10-01       ILD  H351.00                 Bagassosis   
1     ID_0000     2003-10-01    Wheeze  1737.11          Wheezing symptoms   
2     ID_0001     1991-03-19       ILD  H563200         Pulmonary fibrosis   
3     ID_0001     1991-03-19       ILD  H350.00              Farmer’s lung   
4     ID_0001     1991-03-19    Asthma  663V000          Occasional asthma   
...       ...            ...       ...      ...                        ...   
7541  ID_2997     2005-10-09    Asthma  173A.00    Exercise induced asthma   
7542  ID_2997     2005-10-09      COPD     J410  Simple chronic bronchitis   
7543  ID_2997     2005-10-09      COPD  66Yg.00        COPD disturbs sleep   
7544  ID_2998     1992-07-26    Wheeze  1737.11          Wheezing symptoms   
7545  ID_2999     2006-11-08      COPD     J43X                  Emphysema   

     Event_date  
0    2004-04-22  
1    2004-06-11  
2    1993

In [181]:
# Convert DataFrame to TSV format and save to a specific path
conditionEvent.to_csv('C:/Users/User/Desktop/conditionEvent_synthetic_data.tsv', sep='\t', index=False)

measurement


In [182]:
##contains BMI, body height, and smoking status sub section
##main column contains PPID, measurement date, code, textval numeric val
#move PPID and reg start date from initial dataframe

measurement = conditionEvent[['PPID','Reg_Start_Date']]
print(measurement)

         PPID Reg_Start_Date
0     ID_0000     2003-10-01
1     ID_0000     2003-10-01
2     ID_0001     1991-03-19
3     ID_0001     1991-03-19
4     ID_0001     1991-03-19
...       ...            ...
7541  ID_2997     2005-10-09
7542  ID_2997     2005-10-09
7543  ID_2997     2005-10-09
7544  ID_2998     1992-07-26
7545  ID_2999     2006-11-08

[7546 rows x 2 columns]


In [186]:
##reduce all replicated PPID more than 3 to 3

# Reduce occurrences of each PPID to a maximum of 3
def limit_ppid_occurrences(df, max_occurrences=3): #looks through all the lines for max occurence
    return df.groupby('PPID').head(max_occurrences).reset_index(drop=True) #groups by 3 and drops more than

measurement = limit_ppid_occurrences(measurement) #apply function to the df
print(measurement)

         PPID Reg_Start_Date            code
0     ID_0000     2003-10-01    BODY HEIGHTS
1     ID_0000     2003-10-01    BODY HEIGHTS
2     ID_0001     1991-03-19             BMI
3     ID_0001     1991-03-19             BMI
4     ID_0001     1991-03-19             BMI
...       ...            ...             ...
6751  ID_2997     2005-10-09    BODY HEIGHTS
6752  ID_2997     2005-10-09    BODY HEIGHTS
6753  ID_2997     2005-10-09    BODY HEIGHTS
6754  ID_2998     1992-07-26             BMI
6755  ID_2999     2006-11-08  SMOKING STATUS

[6756 rows x 3 columns]


In [187]:
# Text codes
text_codes = ['BMI', 'BODY HEIGHTS', 'SMOKING STATUS']

# Create a function to assign text codes
#Function ensures each PPID gets its code
def assign_text_codes(measurement, text_code_list):
    text_code_dict = {}
    for ppid in measurement['PPID'].unique():
        np.random.shuffle(text_code_list)
        text_code_dict[ppid] = text_code_list.copy()
    
    text_codes_assigned = []
    for ppid in measurement['PPID']:
        text_codes_assigned.append(text_code_dict[ppid].pop()) #create a list of the text code and ensure each row has 1 of each while checking for PPID
    return text_codes_assigned

# Assign the text codes 
measurement['text_code'] = assign_text_codes(measurement, text_codes)

print(measurement)

         PPID Reg_Start_Date            code       text_code
0     ID_0000     2003-10-01    BODY HEIGHTS    BODY HEIGHTS
1     ID_0000     2003-10-01    BODY HEIGHTS             BMI
2     ID_0001     1991-03-19             BMI  SMOKING STATUS
3     ID_0001     1991-03-19             BMI             BMI
4     ID_0001     1991-03-19             BMI    BODY HEIGHTS
...       ...            ...             ...             ...
6751  ID_2997     2005-10-09    BODY HEIGHTS    BODY HEIGHTS
6752  ID_2997     2005-10-09    BODY HEIGHTS             BMI
6753  ID_2997     2005-10-09    BODY HEIGHTS  SMOKING STATUS
6754  ID_2998     1992-07-26             BMI    BODY HEIGHTS
6755  ID_2999     2006-11-08  SMOKING STATUS             BMI

[6756 rows x 4 columns]


In [188]:
#drop code column
measurement.drop(['code'], axis=1,inplace=True)
print(measurement)

         PPID Reg_Start_Date       text_code
0     ID_0000     2003-10-01    BODY HEIGHTS
1     ID_0000     2003-10-01             BMI
2     ID_0001     1991-03-19  SMOKING STATUS
3     ID_0001     1991-03-19             BMI
4     ID_0001     1991-03-19    BODY HEIGHTS
...       ...            ...             ...
6751  ID_2997     2005-10-09    BODY HEIGHTS
6752  ID_2997     2005-10-09             BMI
6753  ID_2997     2005-10-09  SMOKING STATUS
6754  ID_2998     1992-07-26    BODY HEIGHTS
6755  ID_2999     2006-11-08             BMI

[6756 rows x 3 columns]


In [192]:
##create a function based on text value
def assign_text_value(row):
    if row['text_code']=='BMI':
        return np.random.choice(['normal', 'underweight', 'obessed', 'overweight'])
    elif row['text_code'] == 'BODY HEIGHTS':
        return np.random.choice(['child_at_measurement', '_'])
    elif row['text_code'] == 'SMOKING STATUS':
        return np.random.choice(['current_smoker', 'ex_smoker', 'none_smoker'])
    return np.Nan

#assign the value to the df
measurement['text_val'] = measurement.apply(assign_text_value,axis=1)
print(measurement)

         PPID Reg_Start_Date       text_code              text_val
0     ID_0000     2003-10-01    BODY HEIGHTS                     _
1     ID_0000     2003-10-01             BMI            overweight
2     ID_0001     1991-03-19  SMOKING STATUS             ex_smoker
3     ID_0001     1991-03-19             BMI           underweight
4     ID_0001     1991-03-19    BODY HEIGHTS  child_at_measurement
...       ...            ...             ...                   ...
6751  ID_2997     2005-10-09    BODY HEIGHTS  child_at_measurement
6752  ID_2997     2005-10-09             BMI                normal
6753  ID_2997     2005-10-09  SMOKING STATUS        current_smoker
6754  ID_2998     1992-07-26    BODY HEIGHTS                     _
6755  ID_2999     2006-11-08             BMI           underweight

[6756 rows x 4 columns]


In [194]:
#assign num_value of the BMI, HEIGHT and smoking status
def assign_num_val(row):
    if row['text_val']=='normal':
        return round(np.random.uniform(18.5,24.9),1) #generates a uniqe number within range
    elif row['text_val'] == 'obessed':
        return round(np.random.uniform(30.0,39.0),1)
    elif row['text_val'] == 'overweight':
        return round(np.random.uniform(25.0,29.9),1)
    elif row['text_val'] == 'underweight':
        return round(np.random.uniform(16.0,18.5),1)
    elif row['text_val'] == 'current_smoker':
        return 1
    elif row['text_val'] == 'ex_smoker':
        return 2
    elif row['text_val'] == 'none_smoker':
        return 0
    elif row['text_val'] == 'child_at_measurement':
        return round(np.random.uniform(42.0,63.5),1)
    elif row['text_val'] == '-':
        return '_'
    return np.nan

##apply function

measurement['num_val'] = measurement.apply(assign_num_val, axis=1)
print(measurement)


         PPID Reg_Start_Date       text_code              text_val  num_val
0     ID_0000     2003-10-01    BODY HEIGHTS                     _      NaN
1     ID_0000     2003-10-01             BMI            overweight     26.5
2     ID_0001     1991-03-19  SMOKING STATUS             ex_smoker      2.0
3     ID_0001     1991-03-19             BMI           underweight     16.2
4     ID_0001     1991-03-19    BODY HEIGHTS  child_at_measurement     62.9
...       ...            ...             ...                   ...      ...
6751  ID_2997     2005-10-09    BODY HEIGHTS  child_at_measurement     43.3
6752  ID_2997     2005-10-09             BMI                normal     22.4
6753  ID_2997     2005-10-09  SMOKING STATUS        current_smoker      1.0
6754  ID_2998     1992-07-26    BODY HEIGHTS                     _      NaN
6755  ID_2999     2006-11-08             BMI           underweight     18.0

[6756 rows x 5 columns]


In [201]:
#measurement date
#after reg start date
#create a function that assign measurement date either on reg start date or 1 month after

def assign_measurement_date(row):
    reg_start = row['Reg_Start_Date']
    #calculate 1 month after reg date for each colum
    measurement_date = reg_start + timedelta(days=30)
    return measurement_date

#apply the function

measurement['Measurement_Date'] = measurement.apply(assign_measurement_date, axis=1)
print(measurement)

         PPID Reg_Start_Date       text_code              text_val  num_val  \
0     ID_0000     2003-10-01    BODY HEIGHTS                     _      NaN   
1     ID_0000     2003-10-01             BMI            overweight     26.5   
2     ID_0001     1991-03-19  SMOKING STATUS             ex_smoker      2.0   
3     ID_0001     1991-03-19             BMI           underweight     16.2   
4     ID_0001     1991-03-19    BODY HEIGHTS  child_at_measurement     62.9   
...       ...            ...             ...                   ...      ...   
6751  ID_2997     2005-10-09    BODY HEIGHTS  child_at_measurement     43.3   
6752  ID_2997     2005-10-09             BMI                normal     22.4   
6753  ID_2997     2005-10-09  SMOKING STATUS        current_smoker      1.0   
6754  ID_2998     1992-07-26    BODY HEIGHTS                     _      NaN   
6755  ID_2999     2006-11-08             BMI           underweight     18.0   

     Measurement_Date  
0          2003-10-31  
1  

In [202]:
#convert to .tsv
measurement.to_csv('C:/Users/User/Desktop/measuremnt_synthetic_data.tsv', sep='\t', index=False)

medication